In [1]:
import argparse
import xarray as xr
import numpy as np
from mlflux.datafunc import load_psd
from mlflux.gotm import read2010
import os

ows_papa_path = '/home/jw8736/code-5.2.1/cases/ows_papa/' # path where ows_papa distribution can be read
filepath = '../data/PSD/fluxes_all_cruises_compilation.nc' # absolute or relative path to the original PSD d/ataset

##### This function computes weights based on three variables: U, To-Ta, RH. Can be expanded to more flexible choices. #######
from adapt.instance_based import KLIEP
def weighting (ds_sample, ds_target, vars_list = ['U','tdiff','rh']):
    # Createan array by stacking the specified variables
    X_sample = np.hstack([np.array(ds_sample[var]).reshape(-1, 1) for var in vars_list])
    X_target = np.hstack([np.array(ds_target[var]).reshape(-1, 1) for var in vars_list])
    kliep = KLIEP(kernel="rbf", gamma=[10**(i-3) for i in range(7)], random_state=0)
    kliep_weights = kliep.fit_weights(X_sample, X_target)
    ds_sample['weight'] = (['time'], kliep_weights)  # Mean of weights is 1
    return ds_sample

2024-10-11 15:49:17.814840: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 15:49:18.742868: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 15:49:18.898533: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 15:49:18.909629: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 15:49:18.975026: I tensorflow/core/platform/cpu_feature_guar

In [11]:
class Arguments:
    def __init__(self, bulkalg, weight, wave, outpath):
        self.bulkalg = bulkalg
        self.weight = weight
        self.wave = wave
        self.outpath = outpath

args = Arguments('coare3p0', 0, 0, '../data/Processed/')

In [12]:
ds_psd = load_psd(filepath, algo=args.bulkalg)
ds_psd['tauby'] = xr.zeros_like(ds_psd['taubx']) # bulk formula doesn't predict tauy

# Optional weighting based on different target
if args.weight == 1:
    # Read in target files
    df = read2010(ows_papa_path, datetimeformat='%Y-%m-%d %H:%M:%S') 
    df_ = df.set_index('datetime'); ds = xr.Dataset.from_dataframe(df_)
    ds_papa = ds.sel(datetime=slice('2012-01-01','2012-12-31'))
    # Adjust variable names
    ds_papa['U'] = ((ds_papa.ux)**2+(ds_papa.uy)**2)**0.5
    ds_papa['tdiff'] = ds_papa.sst - ds_papa.t
    ds_psd['tdiff'] = ds_psd.tsea - ds_psd.tair
    # Compute weights
    ds_psd = weighting (ds_psd, ds_papa, vars_list = ['U','tdiff','rh'])       
elif args.weight == 2:
    print('Global weighting not yet implemented!')       
else:
    print('No weighting!')
          
# Same to desired directory and name
if not os.path.exists(args.outpath):
    # Create the directory and any necessary parent directories
    os.makedirs(args.outpath)
    print(f"Directory {args.outpath} created.")
else:
    print(f"Directory {args.outpath} already exists.")

ds_psd.attrs['bulkalg'] = args.bulkalg
ds_psd.attrs['weight'] = args.weight
ds_psd.attrs['wave'] = args.wave
ds_psd.to_netcdf(args.outpath + f'psd_{args.bulkalg}_weight{args.weight}_wave{args.wave}.nc')

/ext3/miniconda3/lib/python3.11/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/ext3/miniconda3/lib/python3.11/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(


Number of samples: 10079
No weighting!
Directory ../data/Processed/ already exists.


/ext3/miniconda3/lib/python3.11/site-packages/aerobulk/flux.py:280: UserWarning: Checking for misaligned nans and values outside of the valid range is performed by default, but reduces performance. 
If you are sure your data is valid you can deactivate these checks by setting `input_range_check=False`
  warnings.warn(performance_msg)
/ext3/miniconda3/lib/python3.11/site-packages/xarray/coding/times.py:618: RuntimeWarning: invalid value encountered in cast
  int_num = np.asarray(num, dtype=np.int64)


In [13]:
ds = xr.load_dataset('/home/jw8736/mlflux/data/Processed/psd_coare3p0_weight0_wave0.nc')

/ext3/miniconda3/lib/python3.11/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
